In [1]:
from numpy import mean
from numpy import std
from numpy import dstack
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import *
import numpy as np
from tensorflow.keras.regularizers import l2

from sklearn.model_selection import train_test_split
import tensorflow as tf

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

4 Physical GPUs, 1 Logical GPU


In [3]:
s25 = np.load('s25.npy')
h25 = np.load('h25.npy')

In [4]:
s25_labels = np.array([0 for _ in range(0,len(s25))])
h25_labels = np.array([1 for _ in range(0,len(h25))])

In [5]:
X = np.append(s25,h25,axis=0)

In [6]:
y = np.append(s25_labels,h25_labels,axis=0)

In [7]:
X.shape

(1142, 6250, 19)

In [8]:
def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 10, 32
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, input_shape=(n_timesteps,n_features),kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(LeakyReLU())
    model.add(Conv1D(filters=64, kernel_size=3,kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(LeakyReLU())
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100,kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(LeakyReLU())
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid',kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01) ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy',Precision(),Recall(),AUC()])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy,precision,recall,auc = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy,precision,recall,auc

In [9]:
def summarize_results(accuracies,precisions,recalls,aucs):
    m, s = mean(accuracies), std(accuracies)
    print( ' Accuracy: %.3f%% (+/-%.3f) ' % (m, s))
    m, s = mean(precisions), std(precisions)
    print( ' Precision: %.3f%% (+/-%.3f) ' % (m, s))
    m, s = mean(recalls), std(recalls)
    print( ' Recall: %.3f%% (+/-%.3f) ' % (m, s))
    m, s = mean(aucs), std(aucs)
    print( ' AUC: %.3f%% (+/-%.3f) ' % (m, s))

In [10]:
def run_experiment(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    
    testX -= mean
    testX /= std
    # repeat experiment
    scores = list()
    accuracies = list()
    precisions = list()
    recalls = list()
    aucs = list()
    for r in range(repeats):
        accuracy,precision,recall,auc = evaluate_model(trainX, trainy, testX, testy)
        accuracy = accuracy * 100.0
        precision = precision * 100.0
        recall = recall * 100.0
        auc = auc * 100.0
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        aucs.append(auc)
    # summarize results
    summarize_results(accuracies,precisions,recalls,aucs)

In [11]:
run_experiment(X,y)

Epoch 1/10
29/29 [==============================] - 1s 22ms/step - loss: 14.0170 - accuracy: 0.6046 - precision: 0.5866 - recall: 0.5304 - auc: 0.6303
Epoch 2/10
29/29 [==============================] - 1s 21ms/step - loss: 4.5821 - accuracy: 0.8543 - precision: 0.8589 - recall: 0.8248 - auc: 0.9331
Epoch 3/10
29/29 [==============================] - 1s 21ms/step - loss: 3.0724 - accuracy: 0.9693 - precision: 0.9608 - recall: 0.9743 - auc: 0.9970
Epoch 4/10
29/29 [==============================] - 1s 22ms/step - loss: 2.1551 - accuracy: 0.9978 - precision: 0.9953 - recall: 1.0000 - auc: 1.0000
Epoch 5/10
29/29 [==============================] - 1s 21ms/step - loss: 1.6503 - accuracy: 0.9989 - precision: 1.0000 - recall: 0.9977 - auc: 1.0000
Epoch 6/10
29/29 [==============================] - 1s 22ms/step - loss: 1.3622 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - auc: 1.0000
Epoch 7/10
29/29 [==============================] - 1s 22ms/step - loss: 1.2260 - accuracy: 0.9967 

In [12]:
def evaluate_model_2(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 10, 32
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3,input_shape=(n_timesteps,n_features),kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(LeakyReLU())
    model.add(Dropout(0.5))
    model.add(Conv1D(filters=64, kernel_size=3,kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(Dropout(0.5))
    model.add(Conv1D(filters=64, kernel_size=3,kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(LeakyReLU())
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100,kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(LeakyReLU())
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid',kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01) ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy',Precision(),Recall(),AUC()])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy,precision,recall,auc = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy,precision,recall,auc

In [13]:
def run_experiment_2(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    
    testX -= mean
    testX /= std
    # repeat experiment
    scores = list()
    accuracies = list()
    precisions = list()
    recalls = list()
    aucs = list()
    for r in range(repeats):
        accuracy,precision,recall,auc = evaluate_model_2(trainX, trainy, testX, testy)
        accuracy = accuracy * 100.0
        precision = precision * 100.0
        recall = recall * 100.0
        auc = auc * 100.0
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        aucs.append(auc)
    # summarize results
    summarize_results(accuracies,precisions,recalls,aucs)

In [14]:
run_experiment_2(X,y)

Epoch 1/10
29/29 [==============================] - 1s 25ms/step - loss: 47.8187 - accuracy: 0.5115 - precision_5: 0.4749 - recall_5: 0.3972 - auc_5: 0.5126
Epoch 2/10
29/29 [==============================] - 1s 25ms/step - loss: 8.7347 - accuracy: 0.5827 - precision_5: 0.5808 - recall_5: 0.3949 - auc_5: 0.6113
Epoch 3/10
29/29 [==============================] - 1s 25ms/step - loss: 6.5183 - accuracy: 0.7371 - precision_5: 0.7448 - recall_5: 0.6682 - auc_5: 0.8357
Epoch 4/10
29/29 [==============================] - 1s 24ms/step - loss: 5.9090 - accuracy: 0.8028 - precision_5: 0.7616 - recall_5: 0.8435 - auc_5: 0.8853
Epoch 5/10
29/29 [==============================] - 1s 23ms/step - loss: 5.1896 - accuracy: 0.8269 - precision_5: 0.8277 - recall_5: 0.7967 - auc_5: 0.9223
Epoch 6/10
29/29 [==============================] - 1s 25ms/step - loss: 4.3797 - accuracy: 0.9179 - precision_5: 0.9002 - recall_5: 0.9276 - auc_5: 0.9800
Epoch 7/10
29/29 [==============================] - 1s 25ms/ste

In [15]:
def evaluate_model_3(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 10, 32
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3,kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), input_shape=(n_timesteps,n_features)))
    model.add(LeakyReLU())
    model.add(Conv1D(filters=64,kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), kernel_size=3))
    model.add(LeakyReLU())
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100,kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(LeakyReLU())
    model.add(Dropout(0.25))
    model.add(Dense(50,kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(LeakyReLU())
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid',kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01) ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy',Precision(),Recall(),AUC()])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy,precision,recall,auc = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy,precision,recall,auc

In [16]:
def run_experiment_3(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    
    testX -= mean
    testX /= std
    # repeat experiment
    scores = list()
    accuracies = list()
    precisions = list()
    recalls = list()
    aucs = list()
    for r in range(repeats):
        accuracy,precision,recall,auc = evaluate_model_3(trainX, trainy, testX, testy)
        accuracy = accuracy * 100.0
        precision = precision * 100.0
        recall = recall * 100.0
        auc = auc * 100.0
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        aucs.append(auc)
    # summarize results
    summarize_results(accuracies,precisions,recalls,aucs)

In [17]:
run_experiment_3(X,y)

Epoch 1/10
29/29 [==============================] - 1s 20ms/step - loss: 10.8997 - accuracy: 0.5926 - precision_10: 0.5676 - recall_10: 0.5491 - auc_10: 0.6009
Epoch 2/10
29/29 [==============================] - 1s 20ms/step - loss: 6.7634 - accuracy: 0.9189 - precision_10: 0.9041 - recall_10: 0.9252 - auc_10: 0.9727
Epoch 3/10
29/29 [==============================] - 1s 20ms/step - loss: 4.8172 - accuracy: 0.9869 - precision_10: 0.9771 - recall_10: 0.9953 - auc_10: 0.9988
Epoch 4/10
29/29 [==============================] - 1s 20ms/step - loss: 3.4409 - accuracy: 0.9880 - precision_10: 0.9883 - recall_10: 0.9860 - auc_10: 0.9977
Epoch 5/10
29/29 [==============================] - 1s 20ms/step - loss: 2.6916 - accuracy: 0.9978 - precision_10: 0.9977 - recall_10: 0.9977 - auc_10: 1.0000
Epoch 6/10
29/29 [==============================] - 1s 20ms/step - loss: 2.1587 - accuracy: 1.0000 - precision_10: 1.0000 - recall_10: 1.0000 - auc_10: 1.0000
Epoch 7/10
29/29 [===========================